# Start of modelling

In [33]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import GradientBoostingRegressor
from group_lasso import GroupLasso


In [8]:
pip install group_lasso


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Loading data and changing it to be usable

In [5]:
# Read datafile
y = pd.read_csv('Dependent_y.csv', header=0, index_col=0)
X = pd.read_csv('Features_X.csv', header=0, index_col=0)

y.fillna(0, inplace=True)
y.index = pd.to_datetime(y.index)
X.index = pd.to_datetime(X.index)

weights = pd.read_csv('Stocks_weights.csv', header=0)
weights.index = weights['Date']
weights = weights.drop('Date', axis=1)
weights.index = pd.to_datetime(weights.index)

In [28]:
# For small dataset to test methods
eight_pred = ['mvel1', 'beta', 'betasq', 'chmom', 'dolvol', 'idiovol', 'indmom', 'mom1m']
X_eight_pred = X[eight_pred]

## Functions

In [102]:
from sklearn.model_selection import ParameterGrid

def R_oos(num, den):
    R_oos_val = 1 - (np.sum(num)/np.sum(den))
    return R_oos_val

def val_fun(model, params: dict, X_trn, y_trn, X_vld, y_vld, max_iter=10, tol=1e-4):
    best_ros = None
    lst_params = list(ParameterGrid(params))
    no_improvement_count = 0
    for param in lst_params:
        if best_ros == None:
            mod = model().set_params(**param).fit(X_trn, y_trn)
            y_pred = mod.predict(X_vld)
            best_ros = R_oos(y_vld, y_pred)
            best_param = param
        else:
            mod = model().set_params(**param).fit(X_trn, y_trn)
            y_pred = mod.predict(X_vld)
            ros = R_oos(y_vld, y_pred)
            if ros > best_ros:
                best_ros = ros
                best_param = param
                no_improvement_count = 0
            else:
                no_improvement_count += 1
                if no_improvement_count >= max_iter:
                    break
            if abs(ros - best_ros) < tol:
                break
    return best_param

## 1 - OLS(-3)

### Regression with expanding window and with and without Huber loss

In [8]:
def expanding_regression_OLS(Dependent, Predictors, stock_weights, loss = 'OLS', initial_train_years = 18, validation_years = 12, test_years = 1):
    r_port_difference_list = []
    r_port_actual_list =[]
    years = Dependent.index.year.unique()

    for i in range(len(years) - initial_train_years - validation_years):
        start_year = years[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()
           
        if loss == 'OLS':
            model = LinearRegression()
        elif loss == 'Huber':
            model = HuberRegressor(epsilon = 99.9) # Set the epsilon to 99.9%.
        else:
            raise ValueError("Invlaid loss function. Use OLS or Huber.")
        
        # Training the model
        OLS3 = model.fit(X_train, y_train)

        # Predict returns at stock level
        r_stock_pred = OLS3.predict(X_test)
        
        # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])

        # Appending to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
    
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)
       
    return Model_Roos

In [9]:
#OUT-OF-SAMPLE R^2 for OLS-3
OLS_3pred_Roos=expanding_regression_OLS(y,X_3pred,stock_weights=weights)
OLS_3pred_Roos

0.007429295085335963

In [18]:
#OUT-OF-SAMPLE R^2 for OLS-3 with Huber Loss function
OLS_3pred_Roos_H = expanding_regression_OLS(y, X_3pred,stock_weights=weights, loss = 'Huber')
OLS_3pred_Roos_H

0.07175829363270425

## 2 - Dimension Reduction: PCR and PLS

### 2.1 - PCR

In [15]:
def pcr(Dependent, Predictors, stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    
    # Lists to save outcomes.
    component_counts = []
    r_port_difference_list = []
    r_port_actual_list = []

    yrs = Dependent.index.year.unique() # List with all the years. (1957-2016)
    n_components_list = range(1, 7) # To determine the number of components that are tested.
    best_components = None # Initialize and later save the best amount of components.
    best_r2 = -np.inf  # Initialize with negative infinity to find the maximum R-squared

    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()

        # Testing the different components in PCA.
        for n_component in n_components_list:

            pca = PCA(n_components=n_component)
            X_train_pca = pca.fit_transform(X_train) 
            X_val_pca = pca.transform(X_val)

            # Fit Linear Regression on the training set
            model = LinearRegression()
            model.fit(X_train_pca, y_train)

            # Predict on the validation set
            y_val_pred = model.predict(X_val_pca)
            r2 = r2_score(y_val, y_val_pred)

            # Update best components if the current number of components yields a higher R-squared
            if r2 > best_r2:
                best_r2 = r2
                best_components = n_component

        # Save the best number of components to the list
        component_counts.append(best_components)

        # Use the best number of components to fit the final model on the combined training and validation sets
        best_pca = PCA(n_components=best_components)
        X_train_pca = best_pca.fit_transform(X_train)
        X_test_pca = best_pca.transform(X_test)
        
        # Best Model
        PCASP500 = LinearRegression()
        PCASP500.fit(X_train_pca, y_train)

        # Predict returns at the stock level
        r_stock_pred = PCASP500.predict(X_test_pca)
        
        # Portfolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])

        # Appending Values to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
        
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)

    return Model_Roos

In [ ]:
PCR_Roo2 = pcr(Dependent=y, Predictors=X_eight_pred, stock_weights=weights)
PCR_Roo2

### 2.2 - PLS

In [56]:
def walk_forward_pls(y_variable, x_variables,stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    
    # Lists to save outcomes.
    r_port_difference_list = []
    r_port_actual_list =[]
    component_counts = []

    years = x_variables.index.year.unique()
    n_components_list = range(1, 7) # To determine the number of components that are tested.
    best_components = None # Initialize and later save the best amount of components.
    best_r2 = -np.inf  # Initialize with negative infinity to find the maximum R-squared

    for i in range(len(years) - initial_train_years - validation_years): # i is 0 to the last start_year which is the last year minus the start training years and validation years.
        start_year = years[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = x_variables[(x_variables.index.year < end_train_year)]
        X_test = x_variables[(x_variables.index.year >= end_validation_year) & (x_variables.index.year < end_test_year)]
        X_val = x_variables[(x_variables.index.year >= end_train_year) & (x_variables.index.year < end_validation_year)]
        y_train = y_variable[(y_variable.index.year < end_train_year)].values.ravel()
        y_test = y_variable[(y_variable.index.year >= end_validation_year) & (y_variable.index.year < end_test_year)].values.ravel()
        y_val = y_variable[(y_variable.index.year >= end_train_year) & (y_variable.index.year < end_validation_year)].values.ravel()

        # Testing the different components in PLS.
        for n_component in n_components_list:

            # Train the model once on the training set
            pls = PLSRegression(n_components=n_component)
            pls.fit(X_train, y_train) 

           # Predict on the validation set
            y_val_pred = pls.predict(X_val)
            r2 = r2_score(y_val, y_val_pred)

            # Update best components if the current number of components yields a higher R-squared
            if r2 > best_r2:
                best_r2 = r2
                best_components = n_component

        # Save the best number of components to the list
        component_counts.append(best_components)

        # Use the best number of components to fit the final model 
        best_pls = PLSRegression(n_components=best_components)
        best_pls.fit(X_train, y_train)

        # Evaluate the final model on the test set
        r_stock_pred = best_pls.predict(X_test)
        
        # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred.T, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
       
        # Appending to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
    
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)
       
    return Model_Roos

In [57]:
# Runs PLS and saves the results
r_squared_scores_pls = walk_forward_pls(y, X_eight_pred,stock_weights=weights ,initial_train_years=18, validation_years=12, test_years=1)

In [58]:
print(r_squared_scores_pls)

-0.05519827043915937


## 3 - Elastic Net & Lasso & Ridge

Elastic Net -> l1_ratio=0.5
Ridge -> l1_ratio=0
Lasso -> l1_ratio=1

### E-net

In [77]:
from sklearn.linear_model import HuberRegressor, ElasticNet
from sklearn.metrics import r2_score
def ENet(Dependent, Predictors, stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    yrs = Dependent.index.year.unique()
    r_port_difference_list = []
    r_port_actual_list = []
    tuning_par = {
        "alpha": np.linspace(1e-1, 1e-4, num=10),
        "l1_ratio": [0.5], "tol":[1e-2]
    } 
    
    # Now the model runs for every time of the 30 splits and for every possible combination of the tuning parameters.
    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()
        
        # This part runs the tuning to find the best combination of the tuning parameters for every split
        best_par = val_fun(ElasticNet, params=tuning_par, X_trn=X_train, y_trn=y_train, X_vld=X_val, y_vld=y_val)
        
        # Now we test the model
        RF_SP500 = ElasticNet(alpha=best_par['alpha'], l1_ratio=best_par['l1_ratio']).fit(X_train, y_train)

        # Calculate R_squared        
        r_stock_pred = RF_SP500.predict(X_test)
        
                # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Appending to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
    
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)
       
    return Model_Roos

In [79]:
ENet = ENet(Dependent=y,Predictors=X_red_rf,stock_weights=weights) 
ENet

0.25721887452371794

### Lasso

In [80]:
# Set l1_ratio = 1 -> Lasso
def Lasso(Dependent, Predictors, stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    yrs = Dependent.index.year.unique()
    r_port_difference_list = []
    r_port_actual_list = []
    tuning_par = {
        "alpha": np.linspace(1e-1, 1e-4, num=10),
        "l1_ratio": [1], "tol":[1e-2]
    } 
    
    # Now the model runs for every time of the 30 splits and for every possible combination of the tuning parameters.
    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()
        
        # This part runs the tuning to find the best combination of the tuning parameters for every split
        best_par = val_fun(ElasticNet, params=tuning_par, X_trn=X_train, y_trn=y_train, X_vld=X_val, y_vld=y_val)
        
        # Now we test the model
        RF_SP500 = ElasticNet(alpha=best_par['alpha'], l1_ratio=best_par['l1_ratio']).fit(X_train, y_train)

        # Calculate R_squared        
        r_stock_pred = RF_SP500.predict(X_test)
        
                # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Appending to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
    
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)
       
    return Model_Roos

In [81]:
Lasso_score = Lasso(Dependent=y,Predictors=X_red_rf,stock_weights=weights) 
Lasso_score

0.25721887452371794

### Ridge

In [82]:
# Set l1_ratio = 0 -> RIDGE

def Ridge(Dependent, Predictors, stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    yrs = Dependent.index.year.unique()
    r_port_difference_list = []
    r_port_actual_list = []
    tuning_par = {
        "alpha": np.linspace(1e-1, 1e-4, num=10),
        "l1_ratio": [1], "tol":[1e-2]
    } 
    
    # Now the model runs for every time of the 30 splits and for every possible combination of the tuning parameters.
    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()
        
        # This part runs the tuning to find the best combination of the tuning parameters for every split
        best_par = val_fun(ElasticNet, params=tuning_par, X_trn=X_train, y_trn=y_train, X_vld=X_val, y_vld=y_val)
        
        # Now we test the model
        RF_SP500 = ElasticNet(alpha=best_par['alpha'], l1_ratio=best_par['l1_ratio']).fit(X_train, y_train)

        # Calculate R_squared        
        r_stock_pred = RF_SP500.predict(X_test)
        
                # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Appending to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
    
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)
       
    return Model_Roos

In [83]:
Ridge_score = Ridge(Dependent=y,Predictors=X_red_rf,stock_weights=weights) 
Ridge_score

0.25721887452371794

### Implementation of Huber-Loss

Huber-Loss-function

In [103]:
def huber_loss(y_val, y_pred, delta):
    error = y_val - y_pred
    is_small_error = np.abs(error) <= delta
    squared_loss = 0.5 * (error ** 2)
    linear_loss = delta * (np.abs(error) - 0.5 * delta)
    return np.where(is_small_error, squared_loss, linear_loss)

In [104]:
def val_fun_with_huber(model, params: dict, X_trn, y_trn, X_vld, y_vld):
    best_ros = None
    lst_params = list(ParameterGrid(params))
    for param in lst_params:
        if best_ros == None:
            mod = model().set_params(**param).fit(X_trn, y_trn)
            y_pred = mod.predict(X_vld)
            smallest_loss = huber_loss(y_vld, y_pred, delta=99.9)
            best_param = param
        else:
            mod = model().set_params(**param).fit(X_trn, y_trn)
            y_pred = mod.predict(X_vld)
            loss = huber_loss(y_vld, y_pred, delta=99.9)
            if loss < smallest_loss:
                smallest_loss = loss
                best_param = param
    return best_param

### E-Net with Huber

In [105]:
def ENet_with_huber(Dependent, Predictors, stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    yrs = Dependent.index.year.unique()
    r_port_difference_list = []
    r_port_actual_list = []
    tuning_par = {
        "alpha": np.linspace(1e-1, 1e-4, num=10),
        "l1_ratio": [0.5], "tol":[1e-2]
    } 
    
    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years
        end_validation_year = end_train_year + validation_years
        end_test_year = end_validation_year + test_years

        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()
        
        best_par = val_fun_with_huber(ElasticNet, params=tuning_par, X_trn=X_train, y_trn=y_train, X_vld=X_val, y_vld=y_val)
        
        RF_SP500 = ElasticNet(alpha=best_par['alpha'], l1_ratio=best_par['l1_ratio']).fit(X_train, y_train)

        # Calculate R_squared        
        r_stock_pred = RF_SP500.predict(X_test)
        
                # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Appending to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
    
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)
       
    return Model_Roos

In [106]:
ENet_huber_scores = ENet_with_huber(Dependent=y, Predictors=X_red_rf, stock_weights=weights)
ENet_huber_scores

0.2450819024981108

## Lasso + H

In [107]:
def Lasso_with_huber(Dependent, Predictors, stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    yrs = Dependent.index.year.unique()
    r_port_difference_list = []
    r_port_actual_list = []
    tuning_par = {
        "alpha": np.linspace(1e-1, 1e-4, num=10),
        "l1_ratio": [1], "tol":[1e-2]
    } 
    
    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years
        end_validation_year = end_train_year + validation_years
        end_test_year = end_validation_year + test_years

        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()
        
        best_par = val_fun_with_huber(ElasticNet, params=tuning_par, X_trn=X_train, y_trn=y_train, X_vld=X_val, y_vld=y_val)
        
        RF_SP500 = ElasticNet(alpha=best_par['alpha'], l1_ratio=best_par['l1_ratio']).fit(X_train, y_train)

        # Calculate R_squared        
        r_stock_pred = RF_SP500.predict(X_test)
        
                # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Appending to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
    
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)
       
    return Model_Roos

In [108]:
Lasso_with_huber_scores = Lasso_with_huber(Dependent=y, Predictors=X_red_rf, stock_weights=weights)
Lasso_with_huber_scores

0.24486347729415947

## Ridge + H

In [109]:
def Ridge_with_huber(Dependent, Predictors, stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    yrs = Dependent.index.year.unique()
    r_port_difference_list = []
    r_port_actual_list = []
    tuning_par = {
        "alpha": np.linspace(1e-1, 1e-4, num=10),
        "l1_ratio": [0], "tol":[1e-2]
    } 
    
    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years
        end_validation_year = end_train_year + validation_years
        end_test_year = end_validation_year + test_years

        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()
        
        best_par = val_fun_with_huber(ElasticNet, params=tuning_par, X_trn=X_train, y_trn=y_train, X_vld=X_val, y_vld=y_val)
        
        RF_SP500 = ElasticNet(alpha=best_par['alpha'], l1_ratio=best_par['l1_ratio']).fit(X_train, y_train)

        # Calculate R_squared        
        r_stock_pred = RF_SP500.predict(X_test)
        
                # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Appending to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
    
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)
       
    return Model_Roos

In [110]:
Ridge_with_huber_scores = Ridge_with_huber(Dependent=y, Predictors=X_red_rf, stock_weights=weights)
Ridge_with_huber_scores

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.382e+02, tolerance: 6.768e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.382e+02, tolerance: 6.768e+00 Linear regression models with null weight for the l1 regularization term are mor

0.24203593905499088

## 4 - GLM


In [14]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [71]:
def GLM(y, X,stock_weights, initial_train_years=18, validation_years=12, test_years=1, supress_warning=True):
    yrs = X.index.year.unique()
    r_port_difference_list = []
    r_port_actual_list = []
    Roos_list = []
    tuning_par = {
    #'knots': [3],
    'group_reg':[1e-4,1e-1],
    'l1_reg': [1e-4,0],
    'groups': [],
    'random_state': [12308]
    }

    #GETTING THE SPLINES
    spline_data = pd.DataFrame(np.ones((X.shape[0],1)),index=X.index,columns=['const'])
    for i in X.columns:
        i_dat = X.loc[:,i]
        i_sqr = i_dat**2
        i_cut, bins = pd.cut(i_dat, 3, right=True, ordered=True, retbins=True)
        i_dum = pd.get_dummies(i_cut)
        for j in np.arange(3):
            i_dum.iloc[:,j] = i_dum.iloc[:,j]*((i_dat-bins[j])**2)
        i_dum.columns = [f"{i}_{k}" for k in np.arange(1,3+1)]
        spline_data = pd.concat((spline_data,i_dat,i_dum),axis=1)


    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = spline_data[(X.index.year < end_train_year)]
        X_test = spline_data[(X.index.year >= end_validation_year) & (X.index.year < end_test_year)]
        X_val = spline_data[(X.index.year >= end_train_year) & (X.index.year < end_validation_year)]
        y_train = y[(y.index.year < end_train_year)].values.ravel()
        y_test = y[(y.index.year >= end_validation_year) & (y.index.year < end_test_year)].values.ravel()
        y_val = y[(y.index.year >= end_train_year) & (y.index.year < end_validation_year)].values.ravel()
        
        groups = [0]+flatten([list(np.repeat(i,3+1))[:] for i in np.arange(1,X.shape[1]+1)])
        tuning_par['groups'] = groups
        # This part runs the tuning to find the best combination of the tuning parameters for every split
        best_par = val_fun(GroupLasso, params=tuning_par, X_trn=X_train, y_trn=y_train, X_vld=X_val, y_vld=y_val)
        GL=GroupLasso(groups=best_par['groups'], group_reg=best_par['group_reg'], l1_reg=best_par['l1_reg'], fit_intercept=False, random_state=best_par['random_state'],supress_warning=True).fit(X_train,y_train)
        #GL=GroupLasso(groups=best_par.groups,group_reg=best_par.lmd,l1_reg=best_par.l1_reg,fit_intercept=False,random_state=best_par.random_state)
        
        # Predict returns at the stock level
        r_stock_pred = GL.predict(X_test)
        
        # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])

        # Appending to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
    
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)
       
    return Model_Roos   

In [72]:
GLM_Roo2 = GLM(y=y, X=X_red_rf, stock_weights=weights)   
GLM_Roo2

/var/folders/lv/4s79bs8j60n07b1d88ptj5l00000gn/T/ipykernel_52862/3791382187.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Date
1957-01-01    0.315774
1957-01-01    0.344997
1957-01-01    0.328586
1957-01-01    0.361710
1957-01-01    0.328800
                ...   
2016-12-01    0.329053
2016-12-01    0.000000
2016-12-01    0.318052
2016-12-01    0.350582
2016-12-01    0.363564
Length: 359357, dtype: float64' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  i_dum.iloc[:,j] = i_dum.iloc[:,j]*((i_dat-bins[j])**2)
/var/folders/lv/4s79bs8j60n07b1d88ptj5l00000gn/T/ipykernel_52862/3791382187.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Date
1957-01-01    0.000000
1957-01-01    0.000000
1957-01-01    0.000000
1957-01-01    0.000000
1957-01-01    0.000000
                ...   
2016-12-01    0.000000

0.2548427126697421

In [75]:
print('GLM Roos^2 for a small subsample of predictons:',GLM_Roo2)

GLM Roos^2 for a small subsample of predictons: 0.2548427126697421


## 5 - Random Forest

#### Selecting Subsamples

In [91]:
rf_pred = ['mom1m', 'dy']
X_red_rf = X[rf_pred]
X_red_rf

,mom1m,dy
Date,,
1957-01-01,-0.440062,0.000000
1957-01-01,-0.414635,0.000000
1957-01-01,-0.428776,0.000000
1957-01-01,-0.400577,0.000000
1957-01-01,-0.428589,0.000000
...,...,...
2016-12-01,-0.428368,-0.995178
2016-12-01,-0.286416,-0.911305
2016-12-01,-0.438039,-0.777349


#### Random Forest Function

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RF
from datetime import datetime

def Random_F(Dependent, Predictors, stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    yrs = Dependent.index.year.unique()
    r_port_difference_list = []
    r_port_actual_list = []
    tuning_par = {
    'n_estimators': [300],
    'max_depth': [3,6],
    'max_features': [30,50,100],
    'random_state': [12308]
    }
    
    # Now the model runs for every time of the 30 splits and for every possible combination of the tuning parameters.
    # In this case is 30*60 = 1800, but only the best R2 for every split are stored. 
    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()
        
        # This part runs the tuning to find the best combination of the tuning parameters for every split
        best_par = val_fun(RF, params=tuning_par, X_trn=X_train, y_trn=y_train, X_vld=X_val, y_vld=y_val)
        
        # Now we test the model
        RF_SP500 = RF(n_estimators=best_par['n_estimators'], max_depth=best_par['max_depth'], max_features=best_par['max_features'], 
               random_state=best_par['random_state']).fit(X_train, y_train)
        
        # Predict returns at the stock level
        r_stock_pred = RF_SP500.predict(X_test)
        
        # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])

        # Appending values to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
        
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)
        
    return Model_Roos

In [ ]:
RF_Roo2 = Random_F(Dependent=y,Predictors=X_red_rf, stock_weights=weights)   
RF_Roo2
#(Running time with 2 predictors: 14 minutes)

### Characteristics of OLS-3
OLS-3 includes size = 'mvel1', Book-to-Market = 'bm', momentum = 'mom12m' Does it also include 'mom1m','mom6m','mom12m','mom36m'?

In [6]:
# Create Dataset for OLS-3 (Need to determine the momentum variable!)
three_predictors = ['mvel1', 'bm','mom12m']
X_3pred = X[three_predictors]

## 6 - Gradient Boosted Regression Trees

In [ ]:
def GBRT(Dependent, Predictors, initial_train_years=18, validation_years=12, test_years=1):
    yrs = data.index.year.unique()
    r_squared_scores = []
    tuning_par = {
    'n_estimators': range(1, 150),
    'max_depth': range(1,5),
    'learning_rate': range(0,1)
    }
    
    # Now the model runs for every time of the 30 splits and for every possible combination of the tuning parameters.
    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        train_data = data[(data.index.year < end_train_year)]
        val_data = data[(data.index.year >= end_train_year) & (data.index.year < end_validation_year)]
        test_data = data[(data.index.year >= end_validation_year) & (data.index.year < end_test_year)]

        X_train = train_data[Predictors]
        X_val = val_data[Predictors]
        X_test = test_data[Predictors]
        y_train = train_data[Dependent].values.ravel()
        y_val = val_data[Dependent].values.ravel()
        y_test = test_data[Dependent].values.ravel()

        # This part runs the tuning to find the best combination of the tuning parameters for every split
        best_par = val_fun(GradientBoostingRegressor, params=tuning_par, X_trn=X_train, y_trn=y_train, X_vld=X_val, y_vld=y_val)
        
        # Now we test the model
        GBRT_SP500 = GradientBoostingRegressor(n_estimators=best_par['n_estimators'], max_depth=best_par['max_depth'], learning_rate=best_par['learning_rate']).fit(X_train, y_train)

        # Calculate R_squared
        y_test_pred = GBRT_SP500.predict(X_test)
        test_r2 = r2_score(y_test, y_test_pred)
        r_squared_scores.append(test_r2)
        
    return r_squared_scores

In [ ]:
# run function and print resulting R2 value
GBRT_R2 = GBRT(Dependent=y_variable,Predictors=all_predictors)   
print(GBRT_R2)
print(np.mean(GBRT_R2)) 

## Additional Method: XGBOOST

In [ ]:
from xgboost import XGBRegressor

In [ ]:
def XGBoost(Dependent, Predictors, stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    yrs = Dependent.index.year.unique()
    Roos_list = []
    tuning_par = {
    'n_estimators': [500,600,800,1000],
    'max_depth': [1,2],
    'random_state': [12308],
    #'learning_rate': [.01]
    }
    
    # Now the model runs for every time of the 30 splits and for every possible combination of the tuning parameters.
    # In this case is 30*60 = 1800, but only the best R2 for every split are stored. 
    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()
        
        # This part runs the tuning to find the best combination of the tuning parameters for every split
        best_par = val_fun(XGBRegressor, params=tuning_par, X_trn=X_train, y_trn=y_train, X_vld=X_val, y_vld=y_val)
        
        # Now we test the model
        XGB = XGBRegressor(n_estimators=best_par['n_estimators'], max_depth=best_par['max_depth']).fit(X_train, y_train)
        
        # Predict returns at the stock level
        r_stock_pred = XGB.predict(X_test)
        
        # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])

        # Calculate Roos
        Roos_val = R_oos(r_portfolio_test, r_portfolio_pred)
        Roos_list.append(Roos_val)
        
    return Roos_list

In [ ]:
XGB_Roo2 = XGBoost(Dependent=y,stock_weights=weights,Predictors=X_red_rf)   
XGB_Roo2

In [ ]:
np.mean(XGB_Roo2)

## Additional Method: BART Model

pip install ISLP

In [ ]:
from ISLP.bart import BART

In [ ]:
def BARTrees(Dependent, Predictors, stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    yrs = Dependent.index.year.unique()
    Roos_list = []
    tuning_par = {
    'num_trees': [100,200,300],
    'burnin': [50,150,200],
    'max_stages': [500,1000,2000]
    }
    
    # Now the model runs for every time of the 30 splits and for every possible combination of the tuning parameters.
    # In this case is 30*60 = 1800, but only the best R2 for every split are stored. 
    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()
        
        # This part runs the tuning to find the best combination of the tuning parameters for every split
        best_par = val_fun(BART, params=tuning_par, X_trn=np.asarray(X_train), y_trn=y_train, X_vld=np.asarray(X_val), y_vld=y_val)
        
        # Now we test the model
        BART_SP500 = BART(num_trees=best_par['num_trees'], burnin=best_par['burnin'], max_stages=best_par['max_stages']).fit(X_train, y_train)
        
        # Predict returns at the stock level
        r_stock_pred = BART_SP500.predict(np.asarray(X_test))
        
        # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])

        # Calculate Roos
        Roos_val = R_oos(r_portfolio_test, r_portfolio_pred)
        Roos_list.append(Roos_val)
        
    return Roos_list

In [ ]:
BART_Roo2 = BARTrees(Dependent=y,Predictors=X_red_rf, stock_weights=weights)   
BART_Roo2

In [ ]:
np.mean(BART_Roo2)

## Additional Method: Bagging

In [3]:
def Bagging(Dependent, Predictors, stock_weights, initial_train_years=18, validation_years=12, test_years=1):
    yrs = Dependent.index.year.unique()
    r_port_difference_list = []
    r_port_actual_list = []
    tuning_par = {
    'n_estimators': [300],
    'max_depth': [3,6],
    'random_state': [12308]
    }
    
    # Now the model runs for every time of the 30 splits and for every possible combination of the tuning parameters.
    # In this case is 30*60 = 1800, but only the best R2 for every split are stored. 
    for i in range(len(yrs) - initial_train_years - validation_years):
        start_year = yrs[i]
        end_train_year = start_year + initial_train_years  # 18 years of training and increasing with 1 year every iteration
        end_validation_year = end_train_year + validation_years  # 12 years of validation
        end_test_year = end_validation_year + test_years  # 1 year of test

        # Creating training, validation and test sets.
        X_train = Predictors[(Predictors.index.year < end_train_year)]
        X_test = Predictors[(Predictors.index.year >= end_validation_year) & (Predictors.index.year < end_test_year)]
        X_val = Predictors[(Predictors.index.year >= end_train_year) & (Predictors.index.year < end_validation_year)]
        y_train = Dependent[(Dependent.index.year < end_train_year)].values.ravel()
        y_test = Dependent[(Dependent.index.year >= end_validation_year) & (Dependent.index.year < end_test_year)].values.ravel()
        y_val = Dependent[(Dependent.index.year >= end_train_year) & (Dependent.index.year < end_validation_year)].values.ravel()
        
        # This part runs the tuning to find the best combination of the tuning parameters for every split
        best_par = val_fun(RF, params=tuning_par, X_trn=X_train, y_trn=y_train, X_vld=X_val, y_vld=y_val)
        
        # Now we test the model
        RF_SP500 = RF(n_estimators=best_par['n_estimators'], max_depth=best_par['max_depth'], max_features=X_train.shape[1], 
               random_state=best_par['random_state']).fit(X_train, y_train)
        
        # Predict returns at the stock level
        r_stock_pred = RF_SP500.predict(X_test)
        
                # Portofolio test
        r_portfolio_test = np.dot(y_test, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Calculate portfolio return prediction using the given stock weights
        r_portfolio_pred = np.dot(r_stock_pred, stock_weights[(stock_weights.index.year >= end_validation_year) & (stock_weights.index.year < end_test_year)])
        
        # Appending to lists
        r_port_difference_list.append((r_portfolio_test-r_portfolio_pred)**2)
        r_port_actual_list.append(r_portfolio_test**2)
    
    # Calculate Roos
    Model_Roos = R_oos(r_port_difference_list, r_port_actual_list)
       
    return Model_Roos

In [ ]:
Bagg_Roo2 = Bagging(Dependent=y,Predictors=X_red_rf,stock_weights=weights)
Bagg_Roo2